In [7]:
# ⬇️ Cell 1: imports & path setup
import sys
from pathlib import Path
import pandas as pd

# allow imports from 1_code/
REPO_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[0]
sys.path.append("../1_code")

from libraries.imports import *  

# Stage 1 entry points
from step1_extractpages import (
    extract_text_and_score_pages,
    get_expanded_page_range,
    crop_pdf_to_range_and_preview
)

# Stage 2A entry point
from stage2A import extract_tables_advanced

# define all our folders
DATA_DIR    = REPO_ROOT / "0_data"
OUTPUT_DIR  = REPO_ROOT / "2_output"
RESULTS_DIR = REPO_ROOT / "2_output" / "results_of_final_pages_of_stage1_in_stage2"
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
TABLES_DIR = REPO_ROOT / "2_output" / "extracted_tables"
TABLES_DIR.mkdir(exist_ok=True, parents=True)


In [8]:
# ⬇️ Cell 2: run Stage 1 and write out the page‐ranges CSV
pdf_path = Path("../0_data/pdfs/01UK_total_HR.pdf") #add your pdf here

# 1) score & pick best pages
df_scores = extract_text_and_score_pages(pdf_path)

# 2) expand to a full range
range_info = get_expanded_page_range(df_scores, pdf_path)
page_list = range_info["final_page_range"]

# 3) write out for posterity / auditing
pages_csv = RESULTS_DIR / "stage1_page_range.csv"
pd.DataFrame({"page_num": page_list}).to_csv(pages_csv, index=False)

print("➡️ Stage 1 found pages:", page_list)
print("   saved to", pages_csv)


➡️ Stage 1 found pages: [120, 121]
   saved to /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/results_of_final_pages_of_stage1_in_stage2/stage1_page_range.csv


In [9]:
# ⬇️ Cell 3: run Stage 2A on the cropped PDF with correct page indices
from tqdm import tqdm
import shutil
# 1) Crop the PDF again (uses page_list from Cell 2) and get its path
cropped_pdf = crop_pdf_to_range_and_preview(pdf_path, page_list)

# 2) Since the cropped PDF now has len(page_list) pages, build local 0-based indices
local_pages = list(range(len(page_list)))
print("🔢 Extracting tables from cropped PDF pages:", local_pages)

# 3) Run the advanced extractor on those local page indices
extracted = extract_tables_advanced(cropped_pdf, local_pages)

# 4) Save each table as CSV, embedding the **original** page numbers in their filenames
count = 0
pdf_stem = pdf_path.stem  # Gets the filename without extension, e.g., "UniCredit"
pdf_table_dir = TABLES_DIR / pdf_stem
pdf_table_dir.mkdir(parents=True, exist_ok=True)

for local_idx, tables_on_page in extracted:
    original_page = page_list[local_idx]  # zero-based original page index
    for tbl_i, (df_tbl, method, conf) in enumerate(tables_on_page, start=1):
        fn = f"page_{original_page+1:03d}_tbl{tbl_i:02d}_{method.replace(' ','')}_conf{int(conf)}.csv"
        df_tbl.to_csv(pdf_table_dir / fn, index=False)
        count += 1

print(f"✅ {count} tables saved under {TABLES_DIR.resolve()}")  

for file in pdf_table_dir.glob("*.csv"):
    parts = file.stem.split("_")
    if len(parts) < 2 or not parts[1].isdigit():
        continue
    page_folder = pdf_table_dir / f"page_{parts[1]}"
    page_folder.mkdir(exist_ok=True)
    shutil.move(str(file), page_folder / file.name)

print(f"✅ Grouped tables by page in: {pdf_table_dir.resolve()}")

 PDF has 220 pages total.
 Attempting to crop pages: [120, 121]

 Page 121 (index 120) preview:
Annual report 2024 - CFE
121
5.2.	 Annex 2: List of references
ESRS 2 BP-1 5 a , ESRS 2 BP-1 5 b i
74
------------------------------------------------------------

 Page 122 (index 121) preview:
Annual report 2024 - CFE
122
5.3.	 Annex 3: List of omitted information
Reference
 
------------------------------------------------------------

 Cropped PDF saved as: /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/0_data/cropped_pdf/01UK_total_HR_c.pdf
🔢 Extracting tables from cropped PDF pages: [0, 1]

Table Extractor: Attempting ADVANCED table extraction from 2 page(s) of '01UK_total_HR_c.pdf': [1, 2] (1-indexed pages)
----------------------------------------------------------------------


Extracting tables (multi-method):   0%|                 | 0/2 [00:00<?, ?page/s]


Processing Page 1 (0-indexed: 0) of '01UK_total_HR_c.pdf':
  Page 1: Trying Camelot (Lattice)...


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


    Camelot (Lattice) found 0 potential structure(s).
  Page 1: Trying Camelot (Stream)...


...

    Camelot (Stream) found 3 potential structure(s).
      PASSED Reportable Table Check (Camelot-S T0): 67r x 4c, avg_len=13
      Added Camelot-S Table 0 (Acc:100.0, WS:2.2, Conf:97.8)
      PASSED Reportable Table Check (Camelot-S T1): 68r x 4c, avg_len=13
      Added Camelot-S Table 1 (Acc:99.6, WS:2.2, Conf:97.4)
      PASSED Reportable Table Check (Camelot-S T2): 57r x 4c, avg_len=12
      Added Camelot-S Table 2 (Acc:100.0, WS:1.3, Conf:98.7)
  Page 1: Trying Tabula-py...
    Tabula-py failed: [Errno 2] JVM DLL not found: /opt/homebrew/Cellar/openjdk/24.0.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib
  Page 1: Found 3 candidate table(s) from various methods. Selecting best per region...
    Grouped into 3 potential unique table regions.
    Region 1: Selected 'Camelot-Stream' (Confidence: 97.76) over 0 other candidate(s) for this region.
    Region 2: Selected 'Camelot-Stream' (Confidence: 97.39) over 0 other candidate(s) for this region.
    Region 3: Selected 'Camelot-S

Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


    Camelot (Lattice) found 0 potential structure(s).
  Page 2: Trying Camelot (Stream)...
    Camelot (Stream) found 1 potential structure(s).
      PASSED Reportable Table Check (Camelot-S T0): 45r x 6c, avg_len=12
      Added Camelot-S Table 0 (Acc:99.6, WS:22.6, Conf:77.0)
  Page 2: Trying Tabula-py...


Extracting tables (multi-method): 100%|█████████| 2/2 [00:02<00:00,  1.24s/page]

    Tabula-py failed: [Errno 2] JVM DLL not found: /opt/homebrew/Cellar/openjdk/24.0.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib
  Page 2: Found 1 candidate table(s) from various methods. Selecting best per region...
    Grouped into 1 potential unique table regions.
    Region 1: Selected 'Camelot-Stream' (Confidence: 77.04) over 0 other candidate(s) for this region.
  SUMMARY for Page 2: Final selection of 1 table(s).
----------------------------------------------------------------------
✅ 4 tables saved under /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/extracted_tables
✅ Grouped tables by page in: /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/extracted_tables/01UK_total_HR


In [10]:
# --- Load PDF list from Excel
excel_path = DATA_DIR / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"

pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)
pdf_paths = pdf_df["pdf_path"].dropna().unique()

In [11]:
# --- Process each PDF
for rel_path in tqdm(pdf_paths, desc="Processing PDFs"):
    try:
        pdf_path = (DATA_DIR / "pdfs"/ rel_path).resolve()
        if not pdf_path.exists():
            print(f"❌ Skipping: {pdf_path.name} (not found)")
            continue

        print(f"\n📄 Processing PDF: {pdf_path.name}")
        # ➤ Stage 1
        df_scores = extract_text_and_score_pages(pdf_path)
        range_info = get_expanded_page_range(df_scores, pdf_path)
        page_list = range_info["final_page_range"]

        if not page_list:
            print("   ⚠️ No valid page range found. Skipping.")
            continue

        result_csv_path = RESULTS_DIR / f"{pdf_path.stem}_stage1_range.csv"
        pd.DataFrame({"page_num": page_list}).to_csv(result_csv_path, index=False)
        print(f"   ➡️ Page list saved to: {result_csv_path}")

        # ➤ Stage 2A
        cropped_pdf = crop_pdf_to_range_and_preview(pdf_path, page_list)
        local_pages = list(range(len(page_list)))
        extracted = extract_tables_advanced(cropped_pdf, local_pages)

        # ➤ Save extracted tables
        pdf_table_dir = TABLES_DIR / pdf_path.stem
        pdf_table_dir.mkdir(parents=True, exist_ok=True)
        count = 0

        for local_idx, tables_on_page in extracted:
            original_page = page_list[local_idx]
            for tbl_i, (df_tbl, method, conf) in enumerate(tables_on_page, start=1):
                fn = f"page_{original_page+1:03d}_tbl{tbl_i:02d}_{method.replace(' ','')}_conf{int(conf)}.csv"
                df_tbl.to_csv(pdf_table_dir / fn, index=False)
                count += 1

        print(f"   ✅ Saved {count} table(s) under {pdf_table_dir}")

        # ➤ Group by page
        for file in pdf_table_dir.glob("*.csv"):
            parts = file.stem.split("_")
            if len(parts) < 2 or not parts[1].isdigit():
                continue
            page_folder = pdf_table_dir / f"page_{parts[1]}"
            page_folder.mkdir(exist_ok=True)
            shutil.move(str(file), page_folder / file.name)

        print(f"   📁 Grouped by page in: {pdf_table_dir}")

    except Exception as e:
        print(f"❌ Error processing {rel_path}: {e}")

Processing PDFs:   0%|                                  | 0/100 [00:00<?, ?it/s]


📄 Processing PDF: 01UK_total_HR.pdf
   ➡️ Page list saved to: /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/results_of_final_pages_of_stage1_in_stage2/01UK_total_HR_stage1_range.csv
 PDF has 220 pages total.
 Attempting to crop pages: [120, 121]

 Page 121 (index 120) preview:
Annual report 2024 - CFE
121
5.2.	 Annex 2: List of references
ESRS 2 BP-1 5 a , ESRS 2 BP-1 5 b i
74
------------------------------------------------------------

 Page 122 (index 121) preview:
Annual report 2024 - CFE
122
5.3.	 Annex 3: List of omitted information
Reference
 
------------------------------------------------------------

 Cropped PDF saved as: /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/0_data/cropped_pdf/01UK_total_HR_c.pdf

Table Extractor: Attempting ADVANCED table extraction from 2 page(s) of '01UK_total_HR_c.pdf': [1, 2] (1-indexed pages)
--------------------------------------


Extracting tables (multi-method):   0%|                 | 0/2 [00:00<?, ?page/s]Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value



Processing Page 1 (0-indexed: 0) of '01UK_total_HR_c.pdf':
  Page 1: Trying Camelot (Lattice)...


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


...

    Tabula-py failed: [Errno 2] JVM DLL not found: /opt/homebrew/Cellar/openjdk/24.0.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib
  Page 3: Found 1 candidate table(s) from various methods. Selecting best per region...
    Grouped into 1 potential unique table regions.
    Region 1: Selected 'Camelot-Stream' (Confidence: 30.71) over 0 other candidate(s) for this region.
  SUMMARY for Page 3: Final selection of 1 table(s).

Processing Page 4 (0-indexed: 3) of 'Zabka-Group-2024-AR-Full-year_c.pdf':
  Page 4: Trying Camelot (Lattice)...
    Camelot (Lattice) found 0 potential structure(s).
  Page 4: Trying Camelot (Stream)...
    Camelot (Stream) found 1 potential structure(s).
      PASSED Reportable Table Check (Camelot-S T0): 35r x 12c, avg_len=20
      Added Camelot-S Table 0 (Acc:100.0, WS:66.7, Conf:33.3)
  Page 4: Trying Tabula-py...



Extracting tables (multi-method):  80%|███████▏ | 4/5 [00:06<00:01,  1.58s/page]

    Tabula-py failed: [Errno 2] JVM DLL not found: /opt/homebrew/Cellar/openjdk/24.0.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib
  Page 4: Found 1 candidate table(s) from various methods. Selecting best per region...
    Grouped into 1 potential unique table regions.
    Region 1: Selected 'Camelot-Stream' (Confidence: 33.29) over 0 other candidate(s) for this region.
  SUMMARY for Page 4: Final selection of 1 table(s).

Processing Page 5 (0-indexed: 4) of 'Zabka-Group-2024-AR-Full-year_c.pdf':
  Page 5: Trying Camelot (Lattice)...
    Camelot (Lattice) found 0 potential structure(s).
  Page 5: Trying Camelot (Stream)...
    Camelot (Stream) found 2 potential structure(s).
      PASSED Reportable Table Check (Camelot-S T0): 35r x 11c, avg_len=24
      Added Camelot-S Table 0 (Acc:97.4, WS:60.8, Conf:36.6)
      PASSED Reportable Table Check (Camelot-S T1): 38r x 11c, avg_len=24
      Added Camelot-S Table 1 (Acc:96.6, WS:61.0, Conf:35.5)
  Page 5: Trying Tabula-py...



Processing PDFs: 100%|████████████████████████| 100/100 [25:58<00:00, 15.58s/it]

    Tabula-py failed: [Errno 2] JVM DLL not found: /opt/homebrew/Cellar/openjdk/24.0.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib
  Page 5: Found 2 candidate table(s) from various methods. Selecting best per region...
    Grouped into 2 potential unique table regions.
    Region 1: Selected 'Camelot-Stream' (Confidence: 36.60) over 0 other candidate(s) for this region.
    Region 2: Selected 'Camelot-Stream' (Confidence: 35.55) over 0 other candidate(s) for this region.
  SUMMARY for Page 5: Final selection of 2 table(s).
----------------------------------------------------------------------
   ✅ Saved 6 table(s) under /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/extracted_tables/Zabka-Group-2024-AR-Full-year
   📁 Grouped by page in: /Users/valeriiaklynna/Documents/GitHub/bachelor-thesis-group32-new/bachelor-thesis-group32-folder/2_output/extracted_tables/Zabka-Group-2024-AR-Full-year
